## Step 1)
Run to install prerequisites 

In [7]:
!pip install ipywidgets
!pip install pandas
!pip install astropy
!pip install numpy
!pip install jupyter_ui_poll

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Step 2)

set the search radius... it should be a small angular separation.

In [8]:
## SETTINGS

search_radius = 1/3600.0 #in degrees




import math
import numpy as np
import pandas as pd
import astropy


import astroquery
from astroquery.ipac.ned import Ned
import pandas as pd

def query_ned_by_name(name):
    result_table = None
    success = False
    msg = "Match Found!"
    try:
        result_table = Ned.query_object(name) #astropy.table.Table
        success = True
    except astroquery.exceptions.RemoteServiceError as error:
        msg = error
    except Exception as error:
        msg = error
    return result_table, success, msg


def query_ned_by_coords(pass_ra, pass_dec, pass_radius): # uses deg
    result_table = None
    success = False
    msg = "Match Found!"
    try:

        position = SkyCoord(ra=pass_ra, dec=pass_dec, unit=(u.deg, u.deg), frame='icrs')
        result_table = Ned.query_region(position, radius=pass_radius * u.deg)
        success = True
    except astroquery.exceptions.RemoteServiceError as error:
        msg = error
    except Exception as error:
        msg = error
    return result_table, success, msg



def is_within_rad(ra_1, ra_2, dec_1, dec_2):
    return (search_rad / 3600.0) > (math.sqrt((math.pow((((ra_1 * (math.pi / 180.0)) - (ra_2 * (math.pi / 180.0))) * (math.cos((dec_1 * (math.pi / 180.0))))), 2)) + math.pow(((dec_1 * (math.pi / 180.0)) - (dec_2 * (math.pi / 180.0))), 2))) * (180.0 / math.pi)



# load csv files
all_surveyed_path = './cleansed_data/mcpikeej_nonmasers_noduplicates_rudmingb_cleansed.csv'
masers_path = './data/MCPMaser2020_rudmingb.csv'

all_surveyed = pd.read_csv(all_surveyed_path)
masers = pd.read_csv(masers_path)





## Step 3)

Prepare the data indicies

## All Surveyed Data Sample: 

In [9]:
print(all_surveyed)

      count2019_01       sourcename          ra        dec   Velo  Tsys  Int  \
0                1   RXSJ00001+0523    0.049083   5.388170  11992  51.3    3   
1                2      KUG2358+330    0.242083  33.343900  12921  36.5    8   
2                3  0001233+4733537    0.347083  47.564900   5237  39.5   10   
3                5         NGC-7806    0.375250  31.441900   4768  42.2    5   
4                6  0001383+2329011    0.409667  23.483600   4382  38.9   10   
...            ...              ...         ...        ...    ...   ...  ...   
4063          6348    MCG-03.01.002  359.199000 -16.509600  10734  52.6   10   
4064          6350        UGC-12864  359.350000  30.992200   4683  44.3    5   
4065          6351    235744+003919  359.432000   0.655194   6975  55.6   10   
4066          6352         UGC12879  359.755000  18.834200   5385  33.1   10   
4067          6353           IC1524  359.795000  -4.126920   5744  43.1   10   

      Rms1   Vlo1   Vhi1  
0     10.9  

## Masers Data Sample: 

In [10]:
print(masers)

     count2019    Source_Name  RA_(J2000)  Dec_(J2000) Vsys_(km/s)   Lum Class
0            1         NGC_23    2.473333    25.923060        4565   180     ?
1            2         NGC_17    2.777083   -12.107220        5931   <10     ?
2            3     J0011-0054    2.938333    -0.908611       14384   527     ?
3            4     J0027+4544    6.855417    45.740830       12003   507     ?
4            5           IC10    5.074584    59.308610        -350  0.02    SF
..         ...            ...         ...          ...         ...   ...   ...
175        176       NGC_7479  346.236100    12.322890        2381    19     ?
176        177        IC_1481  349.854600     5.906111        6120   320     ?
177        178  MCG+05-55-041  354.318500    31.800720        9356    93     ?
178        179       NGC_7738  356.008300     0.516667        6762   468     ?
179        180    CGCG498-038  358.934200    30.212220        9240   268     ?

[180 rows x 7 columns]


## Column Identifiers
Use the data samples to record the relevant column headers 

In [11]:
## create a key-lookup table
# Column identifiers for all_surveyed
all_surveyed_cols = dict(
ra = 'ra',
dec = 'dec',
uid = 'count2019_01',
name = 'sourcename')

# Column identifiers for masers
masers_cols = dict(
ra = 'RA_(J2000)',
dec = 'Dec_(J2000)',
uid = 'count2019',
name = 'Source_Name')

# print(masers_cols["ra"])

## Step 4)

Compare Masers and All-Surveyed by separation.

For any objects where there is more than one match, lets let the user decide.

In [12]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astroquery.ipac.ned import Ned
from astropy.table import Table
from astropy import units as u
from jupyter_ui_poll import ui_events
import sys
from astropy.coordinates import Angle

import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display
from astropy.table import Table
import time


def compare_masers(all_surveyed,masers,all_surveyed_cols,masers_cols):
    
    ## get the column header keys:
    # for all_surveyed ...
    all_ra_key = all_surveyed_cols['ra']
    all_dec_key = all_surveyed_cols['dec']
    all_uid_key = all_surveyed_cols['uid']
    all_name_key = all_surveyed_cols['name']
    # ... and for masers
    mas_ra_key = all_surveyed_cols['ra']
    mas_dec_key = all_surveyed_cols['dec']
    mas_uid_key = all_surveyed_cols['uid']
    mas_name_key = all_surveyed_cols['name']

    # first, make iterable Series for ra, dec, and SkyCoords

    all_ra = all_surveyed[all_ra_key].values
    all_dec = all_surveyed[all_dec_key].values
    all_coords = SkyCoord(all_ra * u.deg, all_dec * u.deg)

    mas_ra = masers[mas_ra_key].values
    mas_dec = masers[mas_dec_key].values
    mas_coords = SkyCoord(mas_ra * u.deg, mas_dec * u.deg)
    
    # put the search radius in degrees
    seplimit = search_radius * u.deg

    ## perform the proximity search with astropy
    idx1, idx2, sep2d, dist3d = astropy.coordinates.search_around_sky(all_coords, mas_coords, seplimit)

    print(idx1)
    print(idx2)




compare_masers(all_surveyed,masers,all_surveyed_cols,masers_cols)

KeyError: 'ra'

##  